In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)
from datetime import timedelta
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
with open('MED_MICU.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
len(set(data['nor_hadm_ids']))

64530

In [4]:
len(data['ab_hadm_ids'])

1030

In [5]:
transfer = pd.read_parquet('transfers.parquet')
transfer = transfer.sort_values(by=['hadm_id', 'intime']).reset_index(drop=True)

In [6]:
transfer_ab = transfer[transfer['hadm_id'].isin(data['ab_hadm_ids'])]
transfer_ab

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
3552,17223646,20023461.0,32836054,ED,Emergency Department,2152-10-31 14:43:00,2152-10-31 21:39:00
3553,17223646,20023461.0,37492428,admit,Medicine,2152-10-31 21:39:00,2152-11-03 11:06:02
3554,17223646,20023461.0,35038340,transfer,Medical Intensive Care Unit (MICU),2152-11-03 11:06:02,2152-11-04 16:22:17
3555,17223646,20023461.0,37626000,transfer,Medical/Surgical Intensive Care Unit (MICU/SICU),2152-11-04 16:22:17,2152-11-12 21:56:52
3556,17223646,20023461.0,34589770,transfer,Hematology/Oncology,2152-11-12 21:56:52,2152-11-13 03:28:15
...,...,...,...,...,...,...,...
1560603,18164304,29997500.0,34987291,ED,Emergency Department,2115-05-12 19:12:00,2115-05-13 03:07:00
1560604,18164304,29997500.0,37369823,admit,Medicine,2115-05-13 03:07:00,2115-05-18 14:02:33
1560605,18164304,29997500.0,34008495,transfer,Medical Intensive Care Unit (MICU),2115-05-18 14:02:33,2115-05-21 20:46:21
1560606,18164304,29997500.0,31432534,transfer,Hematology/Oncology Intermediate,2115-05-21 20:46:21,2115-05-27 18:26:46


In [7]:
admit_df = transfer_ab[transfer_ab['eventtype'] == 'admit']

# Step 2: 각 'hadm_id'에 대해 첫 번째 'intime'을 선택합니다.
admit_times = admit_df.groupby('hadm_id')['intime'].first().reset_index()

# Step 3: 'MICU'를 최종적으로 이용한 경우의 'intime'을 추출합니다.
micu_df = transfer_ab[transfer_ab['careunit'] == 'Medical Intensive Care Unit (MICU)']
micu_first_times = micu_df.groupby('hadm_id').head(1)
micu_times = micu_first_times[['hadm_id', 'intime']]

# Step 4: 결과를 결합합니다.
result_df = pd.merge(admit_times, micu_times, on='hadm_id')
result_df.columns = ['hadm_id', 'intime_admit', 'outtime']


In [9]:
result_df['abnormal']=1
ed_hadm_ids = transfer_ab[transfer_ab['eventtype'] == 'ED']['hadm_id'].unique()

result_df['ED'] = 0

result_df.loc[result_df['hadm_id'].isin(ed_hadm_ids), 'ED'] = 1
result_df

,hadm_id,intime_admit,outtime,abnormal,ED
0,20023461.0,2152-10-31 21:39:00,2152-11-03 11:06:02,1,1
1,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1
2,20034762.0,2164-06-04 12:46:49,2164-06-05 22:28:11,1,0
3,20098037.0,2131-12-15 20:22:00,2131-12-20 22:48:56,1,1
4,20108677.0,2182-08-18 22:43:00,2182-08-19 16:43:58,1,1
...,...,...,...,...,...
1025,29951240.0,2183-09-04 23:09:00,2183-09-05 12:11:12,1,1
1026,29975769.0,2140-07-16 18:11:00,2140-07-17 16:03:39,1,1
1027,29988388.0,2130-08-25 22:07:00,2130-08-27 20:16:04,1,1
1028,29996899.0,2174-07-08 02:51:00,2174-07-10 17:22:25,1,1


In [10]:
transfer_nor = transfer[transfer['hadm_id'].isin(data['nor_hadm_ids'])]
transfer_nor

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10467237,20000019.0,39027130,ED,Emergency Department,2159-03-20 18:56:00,2159-03-20 22:48:00
1,10467237,20000019.0,37986519,admit,Medicine,2159-03-20 22:48:00,2159-03-23 16:54:19
2,10467237,20000019.0,38326374,discharge,None,2159-03-23 16:54:19,None
3,16925328,20000024.0,35408147,ED,Emergency Department,2151-05-25 15:03:00,2151-05-26 00:03:00
4,16925328,20000024.0,33480796,admit,Medicine,2151-05-26 00:03:00,2151-05-26 18:36:48
...,...,...,...,...,...,...,...
1560905,10698563,29999501.0,31048702,admit,Medicine,2135-06-26 17:28:00,2135-06-29 14:46:10
1560906,10698563,29999501.0,33983978,discharge,None,2135-06-29 14:46:10,None
1560931,10382924,29999723.0,39960289,ED,Emergency Department,2170-08-09 15:41:00,2170-08-09 19:50:00
1560932,10382924,29999723.0,31490618,admit,Medicine,2170-08-09 19:50:00,2170-08-10 18:34:46


In [11]:
admit_df = transfer_nor[transfer_nor['eventtype'] == 'admit']

admit_times = admit_df.groupby('hadm_id')['intime'].first().reset_index()

discharge_df = transfer_nor[transfer_nor['eventtype'] == 'discharge']
first_discharge_df = discharge_df.groupby('hadm_id').tail(1)
first_discharge_df = first_discharge_df[['hadm_id', 'intime']]

result_df_1 = pd.merge(admit_times, first_discharge_df, on='hadm_id')
result_df_1.columns = ['hadm_id', 'intime_admit', 'outtime']

In [12]:
result_df_1['abnormal']=0
ed_hadm_ids = transfer_nor[transfer_nor['eventtype'] == 'ED']['hadm_id'].unique()

result_df_1['ED'] = 0

result_df_1.loc[result_df_1['hadm_id'].isin(ed_hadm_ids), 'ED'] = 1
result_df_1

,hadm_id,intime_admit,outtime,abnormal,ED
0,20000019.0,2159-03-20 22:48:00,2159-03-23 16:54:19,0,1
1,20000024.0,2151-05-26 00:03:00,2151-05-26 18:36:48,0,1
2,20000034.0,2174-05-22 19:14:02,2174-05-24 17:27:40,0,0
3,20000057.0,2190-01-15 18:40:00,2190-01-18 16:05:24,0,1
4,20000254.0,2138-07-12 20:50:00,2138-07-14 11:23:46,0,1
...,...,...,...,...,...
64525,29998779.0,2185-12-07 14:34:00,2185-12-09 16:35:00,0,1
64526,29999161.0,2175-03-20 13:54:12,2175-03-21 12:43:43,0,0
64527,29999234.0,2184-05-17 22:45:53,2184-05-18 03:31:56,0,1
64528,29999501.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,1


In [13]:
hos =pd.read_parquet('admissions.parquet')
hos_1=hos[['hadm_id','admittime','dischtime','deathtime','race','discharge_location']]
hos_1

,hadm_id,admittime,dischtime,deathtime,race,discharge_location
0,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,None,WHITE,HOME
1,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,None,WHITE,HOME
2,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,None,WHITE,HOSPICE
3,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,None,WHITE,HOME
4,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,None,WHITE,None
...,...,...,...,...,...,...
431226,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,None,WHITE,HOME HEALTH CARE
431227,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,None,WHITE,HOME HEALTH CARE
431228,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,WHITE,DIED
431229,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,None,WHITE,HOME


In [14]:
mapping = {"WHITE": "White","BLACK/AFRICAN AMERICAN": "Black","OTHER": "Other","HISPANIC/LATINO - PUERTO RICAN": "Other","WHITE - OTHER EUROPEAN": "White","HISPANIC/LATINO - DOMINICAN": "Other","BLACK/CAPE VERDEAN": "Black","ASIAN - CHINESE": "Other","ASIAN": "Other","UNKNOWN": "Other","WHITE - RUSSIAN": "White","BLACK/AFRICAN": "Black","BLACK/CARIBBEAN ISLAND": "Black","HISPANIC OR LATINO": "Other","HISPANIC/LATINO - GUATEMALAN": "Other","ASIAN - ASIAN INDIAN": "Other","ASIAN - SOUTH EAST ASIAN": "Other","HISPANIC/LATINO - SALVADORAN": "Other","WHITE - BRAZILIAN": "White","PORTUGUESE": "Other","WHITE - EASTERN EUROPEAN": "White","HISPANIC/LATINO - COLUMBIAN": "Other","HISPANIC/LATINO - MEXICAN": "Other","SOUTH AMERICAN": "Other","AMERICAN INDIAN/ALASKA NATIVE": "Other","HISPANIC/LATINO - HONDURAN": "Other","HISPANIC/LATINO - CUBAN": "Other","HISPANIC/LATINO - CENTRAL AMERICAN": "Other","ASIAN - KOREAN": "Other","PATIENT DECLINED TO ANSWER": "Other","NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER": "Other","MULTIPLE RACE/ETHNICITY": "Other","UNABLE TO OBTAIN": "Other"}

In [15]:
hos_1['race'] = hos_1['race'].map(mapping)

C:\Users\DAHS\AppData\Local\Temp\ipykernel_2220\2027026405.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hos_1['race'] = hos_1['race'].map(mapping)


In [20]:
nor_hadm_ids = result_df_1['hadm_id']

In [21]:
hos_2 = hos_1[hos_1['hadm_id'].isin(nor_hadm_ids)]
hos_3 = hos_2[~hos_2['discharge_location'].isin(['ACUTE HOSPITAL','HOSPICE','AGAINST ADVICE'])]
hos_4 = hos_3[~hos_3['discharge_location'].isnull()]

In [22]:
hos_4

,hadm_id,admittime,dischtime,deathtime,race,discharge_location
5,23052089,2160-11-21 01:56:00,2160-11-25 14:52:00,None,White,HOME HEALTH CARE
28,20897796,2193-08-15 01:01:00,2193-08-17 15:07:00,None,Black,HOME HEALTH CARE
29,24947999,2190-11-06 20:57:00,2190-11-08 15:58:00,None,Black,HOME HEALTH CARE
30,25242409,2191-04-03 18:48:00,2191-04-11 16:21:00,None,Black,SKILLED NURSING FACILITY
31,25911675,2191-05-23 15:33:00,2191-05-24 17:14:00,None,Black,HOME HEALTH CARE
...,...,...,...,...,...,...
431147,27282608,2177-07-25 04:34:00,2177-07-26 14:07:00,None,Black,HOME HEALTH CARE
431150,21096018,2155-06-03 22:42:00,2155-06-10 20:00:00,None,White,HOME
431166,23946928,2177-03-10 18:11:00,2177-03-12 15:03:00,None,White,HOME
431170,24639135,2165-10-06 17:32:00,2165-10-09 12:50:00,None,White,HOME


In [23]:
result_df_1=result_df_1[result_df_1['hadm_id'].isin(hos_4['hadm_id'])]

In [24]:
result_df_2 = pd.concat([result_df,result_df_1]).reset_index(drop=True)
result_df_2

,hadm_id,intime_admit,outtime,abnormal,ED
0,20023461.0,2152-10-31 21:39:00,2152-11-03 11:06:02,1,1
1,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1
2,20034762.0,2164-06-04 12:46:49,2164-06-05 22:28:11,1,0
3,20098037.0,2131-12-15 20:22:00,2131-12-20 22:48:56,1,1
4,20108677.0,2182-08-18 22:43:00,2182-08-19 16:43:58,1,1
...,...,...,...,...,...
50260,29998554.0,2157-09-30 16:40:00,2157-10-02 16:13:10,0,1
50261,29998779.0,2185-12-07 14:34:00,2185-12-09 16:35:00,0,1
50262,29999161.0,2175-03-20 13:54:12,2175-03-21 12:43:43,0,0
50263,29999501.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,1


In [25]:
hos_11 = hos_1[hos_1['hadm_id'].isin(result_df_2['hadm_id'])][['hadm_id','deathtime','race']]
df = pd.merge(result_df_2,hos_11,how='left',on='hadm_id')

In [26]:
df

,hadm_id,intime_admit,outtime,abnormal,ED,deathtime,race
0,20023461.0,2152-10-31 21:39:00,2152-11-03 11:06:02,1,1,2152-11-16 13:40:00,White
1,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,2186-03-03 10:15:00,White
2,20034762.0,2164-06-04 12:46:49,2164-06-05 22:28:11,1,0,None,Other
3,20098037.0,2131-12-15 20:22:00,2131-12-20 22:48:56,1,1,2131-12-23 08:16:00,White
4,20108677.0,2182-08-18 22:43:00,2182-08-19 16:43:58,1,1,None,White
...,...,...,...,...,...,...,...
50260,29998554.0,2157-09-30 16:40:00,2157-10-02 16:13:10,0,1,None,White
50261,29998779.0,2185-12-07 14:34:00,2185-12-09 16:35:00,0,1,None,White
50262,29999161.0,2175-03-20 13:54:12,2175-03-21 12:43:43,0,0,None,White
50263,29999501.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,1,None,White


### 여기서 admission에서의 intime과 transfer의 intime이 서로 시간의 차이가 평균적으로 1시간 30분 정도 났지만 우리는 내과에 입원후 기준으로 볼 것이기 때문에 transfer time 기준으로 전처리를 진행하였다.

In [27]:
df['intime_admit']=pd.to_datetime(df['intime_admit'])
df['outtime']=pd.to_datetime(df['outtime'])
df['deathtime']=pd.to_datetime(df['deathtime'])

In [28]:
df

,hadm_id,intime_admit,outtime,abnormal,ED,deathtime,race
0,20023461.0,2152-10-31 21:39:00,2152-11-03 11:06:02,1,1,2152-11-16 13:40:00,White
1,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,2186-03-03 10:15:00,White
2,20034762.0,2164-06-04 12:46:49,2164-06-05 22:28:11,1,0,NaT,Other
3,20098037.0,2131-12-15 20:22:00,2131-12-20 22:48:56,1,1,2131-12-23 08:16:00,White
4,20108677.0,2182-08-18 22:43:00,2182-08-19 16:43:58,1,1,NaT,White
...,...,...,...,...,...,...,...
50260,29998554.0,2157-09-30 16:40:00,2157-10-02 16:13:10,0,1,NaT,White
50261,29998779.0,2185-12-07 14:34:00,2185-12-09 16:35:00,0,1,NaT,White
50262,29999161.0,2175-03-20 13:54:12,2175-03-21 12:43:43,0,0,NaT,White
50263,29999501.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,1,NaT,White


In [29]:
df['death'] = ((df['outtime'] > df['deathtime']) & (df['intime_admit'] < df['deathtime'])).astype(int)

In [30]:
df

,hadm_id,intime_admit,outtime,abnormal,ED,deathtime,race,death
0,20023461.0,2152-10-31 21:39:00,2152-11-03 11:06:02,1,1,2152-11-16 13:40:00,White,0
1,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,2186-03-03 10:15:00,White,0
2,20034762.0,2164-06-04 12:46:49,2164-06-05 22:28:11,1,0,NaT,Other,0
3,20098037.0,2131-12-15 20:22:00,2131-12-20 22:48:56,1,1,2131-12-23 08:16:00,White,0
4,20108677.0,2182-08-18 22:43:00,2182-08-19 16:43:58,1,1,NaT,White,0
...,...,...,...,...,...,...,...,...
50260,29998554.0,2157-09-30 16:40:00,2157-10-02 16:13:10,0,1,NaT,White,0
50261,29998779.0,2185-12-07 14:34:00,2185-12-09 16:35:00,0,1,NaT,White,0
50262,29999161.0,2175-03-20 13:54:12,2175-03-21 12:43:43,0,0,NaT,White,0
50263,29999501.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,1,NaT,White,0


In [31]:
df

,hadm_id,intime_admit,outtime,abnormal,ED,deathtime,race,death
0,20023461.0,2152-10-31 21:39:00,2152-11-03 11:06:02,1,1,2152-11-16 13:40:00,White,0
1,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,2186-03-03 10:15:00,White,0
2,20034762.0,2164-06-04 12:46:49,2164-06-05 22:28:11,1,0,NaT,Other,0
3,20098037.0,2131-12-15 20:22:00,2131-12-20 22:48:56,1,1,2131-12-23 08:16:00,White,0
4,20108677.0,2182-08-18 22:43:00,2182-08-19 16:43:58,1,1,NaT,White,0
...,...,...,...,...,...,...,...,...
50260,29998554.0,2157-09-30 16:40:00,2157-10-02 16:13:10,0,1,NaT,White,0
50261,29998779.0,2185-12-07 14:34:00,2185-12-09 16:35:00,0,1,NaT,White,0
50262,29999161.0,2175-03-20 13:54:12,2175-03-21 12:43:43,0,0,NaT,White,0
50263,29999501.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,1,NaT,White,0


In [32]:
df.loc[df['death'] == 1, 'outtime'] = df['deathtime']

In [33]:
df['label'] = ((df['abnormal'] == 1) | (df['death'] == 1)).astype(int)

In [34]:
df.drop(columns='deathtime', inplace=True)
df

,hadm_id,intime_admit,outtime,abnormal,ED,race,death,label
0,20023461.0,2152-10-31 21:39:00,2152-11-03 11:06:02,1,1,White,0,1
1,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,White,0,1
2,20034762.0,2164-06-04 12:46:49,2164-06-05 22:28:11,1,0,Other,0,1
3,20098037.0,2131-12-15 20:22:00,2131-12-20 22:48:56,1,1,White,0,1
4,20108677.0,2182-08-18 22:43:00,2182-08-19 16:43:58,1,1,White,0,1
...,...,...,...,...,...,...,...,...
50260,29998554.0,2157-09-30 16:40:00,2157-10-02 16:13:10,0,1,White,0,0
50261,29998779.0,2185-12-07 14:34:00,2185-12-09 16:35:00,0,1,White,0,0
50262,29999161.0,2175-03-20 13:54:12,2175-03-21 12:43:43,0,0,White,0,0
50263,29999501.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,1,White,0,0


In [35]:
sub_id = hos[['subject_id','hadm_id']]
sub_id

,subject_id,hadm_id
0,10000032,22595853
1,10000032,22841357
2,10000032,25742920
3,10000032,29079034
4,10000068,25022803
...,...,...
431226,19999828,25744818
431227,19999828,29734428
431228,19999840,21033226
431229,19999840,26071774


In [36]:
df

,hadm_id,intime_admit,outtime,abnormal,ED,race,death,label
0,20023461.0,2152-10-31 21:39:00,2152-11-03 11:06:02,1,1,White,0,1
1,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,White,0,1
2,20034762.0,2164-06-04 12:46:49,2164-06-05 22:28:11,1,0,Other,0,1
3,20098037.0,2131-12-15 20:22:00,2131-12-20 22:48:56,1,1,White,0,1
4,20108677.0,2182-08-18 22:43:00,2182-08-19 16:43:58,1,1,White,0,1
...,...,...,...,...,...,...,...,...
50260,29998554.0,2157-09-30 16:40:00,2157-10-02 16:13:10,0,1,White,0,0
50261,29998779.0,2185-12-07 14:34:00,2185-12-09 16:35:00,0,1,White,0,0
50262,29999161.0,2175-03-20 13:54:12,2175-03-21 12:43:43,0,0,White,0,0
50263,29999501.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,1,White,0,0


In [37]:
df = pd.merge(df,sub_id,how='left',on='hadm_id')

In [39]:
df

,hadm_id,intime_admit,outtime,abnormal,ED,race,death,label,subject_id
0,20023461.0,2152-10-31 21:39:00,2152-11-03 11:06:02,1,1,White,0,1,17223646
1,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,White,0,1,16145617
2,20034762.0,2164-06-04 12:46:49,2164-06-05 22:28:11,1,0,Other,0,1,16976998
3,20098037.0,2131-12-15 20:22:00,2131-12-20 22:48:56,1,1,White,0,1,16225551
4,20108677.0,2182-08-18 22:43:00,2182-08-19 16:43:58,1,1,White,0,1,10975446
...,...,...,...,...,...,...,...,...,...
50260,29998554.0,2157-09-30 16:40:00,2157-10-02 16:13:10,0,1,White,0,0,13779116
50261,29998779.0,2185-12-07 14:34:00,2185-12-09 16:35:00,0,1,White,0,0,11163466
50262,29999161.0,2175-03-20 13:54:12,2175-03-21 12:43:43,0,0,White,0,0,16531634
50263,29999501.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,1,White,0,0,10698563


In [38]:
#df.to_csv('pre_hos_v0.1.csv',index=0)

In [207]:
omr=pd.read_parquet('omr.parquet')
omr

,subject_id,chartdate,seq_num,result_name,result_value
0,10000032,2180-04-27,1,Blood Pressure,110/65
1,10000032,2180-04-27,1,Weight (Lbs),94
2,10000032,2180-05-07,1,BMI (kg/m2),18.0
3,10000032,2180-05-07,1,Height (Inches),60
4,10000032,2180-05-07,1,Weight (Lbs),92.15
...,...,...,...,...,...
6439164,19999828,2148-02-26,1,Blood Pressure,115/79
6439165,19999828,2148-04-29,1,Blood Pressure,105/67
6439166,19999828,2148-07-22,1,Blood Pressure,104/76
6439167,19999828,2148-10-19,1,Blood Pressure,112/73


In [236]:
omr.loc[omr['result_name'].str.startswith('Blood Pressure'), 'result_name'] = 'Blood Pressure'
omr.loc[omr['result_name'].str.startswith('BMI'), 'result_name'] = 'BMI'

In [243]:
omr_1=omr[omr['result_name'].isin(['Blood Pressure','BMI'])]

In [247]:
omr_1['chartdate']=pd.to_datetime(omr_1['chartdate'])

C:\Users\DAHS\AppData\Local\Temp\ipykernel_1328\4143770312.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  omr_1['chartdate']=pd.to_datetime(omr_1['chartdate'])


In [250]:
merged_df = pd.merge(df, omr_1, on='subject_id', how='inner')

filtered_omr = merged_df[
    (merged_df['chartdate'] > merged_df['intime_admit']) &
    (merged_df['chartdate'] < merged_df['outtime'])
]


In [261]:
merged_df

,hadm_id,intime_admit,outtime,abnormal,ED,race,death,label,subject_id,chartdate,seq_num,result_name,result_value
0,20023461.0,2152-10-31 21:39:00,2152-11-03 11:06:02,1,1,White,0,1,17223646,2152-11-01,1,BMI,20.6
1,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,White,0,1,16145617,2182-02-03,1,Blood Pressure,95/63
2,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,White,0,1,16145617,2182-02-03,1,BMI,28.7
3,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,White,0,1,16145617,2182-03-07,1,Blood Pressure,116/60
4,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,White,0,1,16145617,2182-03-07,1,BMI,29.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2478512,29999501.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,1,White,0,0,10698563,2138-01-21,1,Blood Pressure,115/43
2478513,29999723.0,2170-08-09 19:50:00,2170-08-10 18:34:46,0,1,White,0,0,10382924,2171-12-21,1,Blood Pressure,128/80
2478514,29999723.0,2170-08-09 19:50:00,2170-08-10 18:34:46,0,1,White,0,0,10382924,2171-12-31,1,Blood Pressure,130/80
2478515,29999723.0,2170-08-09 19:50:00,2170-08-10 18:34:46,0,1,White,0,0,10382924,2172-03-14,1,Blood Pressure,140/100


In [258]:
df['hadm_id'].nunique()

50265

In [269]:
filtered_omr['hadm_id'].nunique()

6397

## omr을 사용하려고 했으나 50265명의 환자중에 6300여명 정도환자만 omr 검사를 했어서 사용을 안하혀고한다. 이유는 2가지 인데 1. 날짜정보만 있다. 2. omr측정지표가 중환자실 사망과 관련이 있지도 않다. 단순 환자의 비율을 비교해 보았을때 

In [40]:
diff=df['outtime']-df['intime_admit']

### 환자의 일반 병동에 있는 describe()

In [41]:
diff.describe([0.01,0.1,0.25,0.5,0.75,0.9,0.99])

count                        50265
mean     4 days 05:23:44.265711727
std      4 days 16:52:30.921178826
min                0 days 00:00:45
1%          0 days 09:52:19.600000
10%         0 days 23:53:42.800000
25%                1 days 19:51:54
50%                2 days 23:31:00
75%                5 days 00:38:28
90%      8 days 00:55:15.599999999
99%        20 days 19:34:07.720000
max              376 days 15:13:58
dtype: object

In [42]:
patient=pd.read_parquet('patients.parquet')
patient

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,None
2,10000068,F,19,2160,2008 - 2010,None
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,None
...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,None
299708,19999829,F,28,2186,2008 - 2010,None
299709,19999840,M,58,2164,2008 - 2010,2164-09-17
299710,19999914,F,49,2158,2017 - 2019,None


In [43]:
patient=patient[['subject_id','gender','anchor_age','anchor_year']]
patient

,subject_id,gender,anchor_age,anchor_year
0,10000032,F,52,2180
1,10000048,F,23,2126
2,10000068,F,19,2160
3,10000084,M,72,2160
4,10000102,F,27,2136
...,...,...,...,...
299707,19999828,F,46,2147
299708,19999829,F,28,2186
299709,19999840,M,58,2164
299710,19999914,F,49,2158


In [44]:
df['admit_year'] = df['intime_admit'].dt.year
df

,hadm_id,intime_admit,outtime,abnormal,ED,race,death,label,subject_id,admit_year
0,20023461.0,2152-10-31 21:39:00,2152-11-03 11:06:02,1,1,White,0,1,17223646,2152
1,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,White,0,1,16145617,2186
2,20034762.0,2164-06-04 12:46:49,2164-06-05 22:28:11,1,0,Other,0,1,16976998,2164
3,20098037.0,2131-12-15 20:22:00,2131-12-20 22:48:56,1,1,White,0,1,16225551,2131
4,20108677.0,2182-08-18 22:43:00,2182-08-19 16:43:58,1,1,White,0,1,10975446,2182
...,...,...,...,...,...,...,...,...,...,...
50260,29998554.0,2157-09-30 16:40:00,2157-10-02 16:13:10,0,1,White,0,0,13779116,2157
50261,29998779.0,2185-12-07 14:34:00,2185-12-09 16:35:00,0,1,White,0,0,11163466,2185
50262,29999161.0,2175-03-20 13:54:12,2175-03-21 12:43:43,0,0,White,0,0,16531634,2175
50263,29999501.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,1,White,0,0,10698563,2135


In [45]:
dfdf = pd.merge(df, patient,  how='left',on='subject_id')
dfdf['age']=dfdf['anchor_age']+dfdf['admit_year']-dfdf['anchor_year']

In [46]:
dfdf

,hadm_id,intime_admit,outtime,abnormal,ED,race,death,label,subject_id,admit_year,gender,anchor_age,anchor_year,age
0,20023461.0,2152-10-31 21:39:00,2152-11-03 11:06:02,1,1,White,0,1,17223646,2152,F,69,2152,69
1,20026981.0,2186-02-25 20:08:00,2186-02-26 21:05:06,1,1,White,0,1,16145617,2186,F,87,2183,90
2,20034762.0,2164-06-04 12:46:49,2164-06-05 22:28:11,1,0,Other,0,1,16976998,2164,F,85,2164,85
3,20098037.0,2131-12-15 20:22:00,2131-12-20 22:48:56,1,1,White,0,1,16225551,2131,F,56,2123,64
4,20108677.0,2182-08-18 22:43:00,2182-08-19 16:43:58,1,1,White,0,1,10975446,2182,M,81,2181,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50260,29998554.0,2157-09-30 16:40:00,2157-10-02 16:13:10,0,1,White,0,0,13779116,2157,M,38,2157,38
50261,29998779.0,2185-12-07 14:34:00,2185-12-09 16:35:00,0,1,White,0,0,11163466,2185,M,35,2185,35
50262,29999161.0,2175-03-20 13:54:12,2175-03-21 12:43:43,0,0,White,0,0,16531634,2175,M,62,2174,63
50263,29999501.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,1,White,0,0,10698563,2135,F,67,2126,76


In [47]:
ward_1=dfdf[['hadm_id','intime_admit','race','gender','age','label']]
ward_2=dfdf[['hadm_id','outtime','race','gender','age','label']]

In [48]:
ward_1.rename(columns={'intime_admit': 'time'}, inplace=True)
ward_2.rename(columns={'outtime': 'time'}, inplace=True)

C:\Users\DAHS\AppData\Local\Temp\ipykernel_2220\1128046633.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ward_1.rename(columns={'intime_admit': 'time'}, inplace=True)
C:\Users\DAHS\AppData\Local\Temp\ipykernel_2220\1128046633.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ward_2.rename(columns={'outtime': 'time'}, inplace=True)


In [49]:
ward=pd.concat([ward_1,ward_2])
ward=ward.sort_values(by=['hadm_id','time'])

In [50]:
ward.reset_index(drop=True,inplace=True)
ward

,hadm_id,time,race,gender,age,label
0,20000019.0,2159-03-20 22:48:00,Other,F,76,0
1,20000019.0,2159-03-23 16:54:19,Other,F,76,0
2,20000024.0,2151-05-26 00:03:00,White,F,92,0
3,20000024.0,2151-05-26 18:36:48,White,F,92,0
4,20000343.0,2137-01-28 01:24:00,Black,M,59,0
...,...,...,...,...,...,...
100525,29999161.0,2175-03-21 12:43:43,White,M,63,0
100526,29999501.0,2135-06-26 17:28:00,White,F,76,0
100527,29999501.0,2135-06-29 14:46:10,White,F,76,0
100528,29999723.0,2170-08-09 19:50:00,White,M,61,0


In [51]:
diag = pd.read_parquet('diagnoses_icd.parquet')
diag_df=diag[['hadm_id','icd_code','icd_version']]
diag_df

,hadm_id,icd_code,icd_version
0,22595853,5723,9
1,22595853,78959,9
2,22595853,5715,9
3,22595853,07070,9
4,22595853,496,9
...,...,...,...
4756321,23865745,41401,9
4756322,23865745,78039,9
4756323,23865745,0413,9
4756324,23865745,36846,9


In [52]:
hypertension_uncomplicated_9=['401']
hypertension_complicated_9=['402','403','404','405']
chf_9=['39891', '40201', '40211','40291', '40401', '40403', '40411', '40413', '40491', '40493', '4254','4255','4256','4257','4258','4259','428']
cardiac_9=['4260', '42613', '4267', '4269', '42610', '42612', '4270','4271','4272','4273','4274','4276','4277','4278','4279','7850', '99601', '99604', 'V450', 'V533']
valvular_disease_9=['0932','394','395','396','397','424','7463','7464','7465','7466', 'V422', 'V433']
Pulmonary_circulation_9=['4150', '4151', '416', '4170', '4178', '4179']
Peripheral_vascular_9=['0930', '4373', '440', '441', '4431','4432',    '4433',    '4434',    '4435',    '4436',    '4437',    '4438',    '4439', '4471', '5571', '5579', 'V434']
Paralysis_9=['3341', '342', '343', '3440','3441','3442','3443','3444','3445','3446', '3449' ]
Other_neurological_disorders_9=['3319', '3320', '3321', '3334', '3335', '33392', '334','335', '3362', '340', '341', '345', '3481', '3483', '7803', '7843']
Chronic_pulmonary_disease_9=['4168', '4169','490', '491', '492', '493', '494', '495', '496', '497', '498', '499', '500', '501', '502', '503', '504', '505','5064', '5081', '5088']
Diabetes_uncomplicated_9=['2500','2501','2502','2503']
Diabetes_complicated_9=['2504','2505','2506','2507','2508','2509']
Hypothyroidism_9=['2409', '243', '244', '2461', '2468']
Renal_failure_9=['40301', '40311', '40391', '40402', '40403', '40412', '40413', '40492', '40493', '585', '586', '5880', 'V420', 'V451', 'V56' ]
Liver_disease_9=['07022', '07023', '07032', '07033', '07044', '07054', '0706', '0709', '4560','4561','4562','570', '571', '5722','5723','5724','5725','5726','5727','5728','5733', '5734', '5738','5739', 'V427']
Pepticulcer_disease_9=['5317', '5319', '5327', '5329','5337', '5339', '5347', '5349']
AIDS_H1V_9=['042','043','044']
Lymphoma_9=['200','201','202','2030', '2386' ]
Metastatic_cancer_9=['196','197','198','199']
Solid_tumor_without_metastasis_9=['140','141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169','170', '171', '172', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195']
Rheumatoid_arthritis_collagen_vascular_diseases_9=['446', '7010', '7101','7102','7103','7104','7108', '7109', '7112', '714','7193', '720', '725', '7285','72889', '72930']
Coagulopathy_9=['286', '2871', '2873','2874','2875']
Obesity_9=['2780']
Weight_loss_9=['260','261','262','263','7832','7994']
Fluid_electrolyte_disorders_9=['2536', '276' ]
Blood_loss_anemia_9=['2800']
Deficiency_anemia_9=['2801','2802','2803','2804','2805','2806','2807','2808','2809', '281']
Alcohol_abuse_9=['2652', '2911','2912','2913','2915','2916','2917','2918','2919','3030','3039', '3050', '3575','4255', '5353','5710','5713', '980','V113']
Drug_abuse_9=['292','304','3052',  '3053',    '3054',    '3055',    '3056',    '3057',    '3058',    '3059','V6542'  ]
Psychoses_9=['2938', '295', '29604', '29614','29644','29654','297','298']
Depression_9=['2962', '2963', '2965', '3004','309', '311' ]

In [53]:
combined_list_9 = (hypertension_uncomplicated_9 +hypertension_complicated_9 +chf_9 +cardiac_9 +valvular_disease_9 +Pulmonary_circulation_9 +Peripheral_vascular_9 +Paralysis_9 +Other_neurological_disorders_9 +Chronic_pulmonary_disease_9 +Diabetes_uncomplicated_9 +Diabetes_complicated_9 +Hypothyroidism_9 +Renal_failure_9 +Liver_disease_9 +Pepticulcer_disease_9 +AIDS_H1V_9 +Lymphoma_9 +Metastatic_cancer_9 +Solid_tumor_without_metastasis_9 +Rheumatoid_arthritis_collagen_vascular_diseases_9 +Coagulopathy_9 +Obesity_9 +Weight_loss_9 +Fluid_electrolyte_disorders_9 +Blood_loss_anemia_9 +Deficiency_anemia_9 +Alcohol_abuse_9 +Drug_abuse_9 +Psychoses_9 +Depression_9)

In [54]:
hypertension_uncomplicated_10=['I10']
hypertension_complicated_10=['I11','I12','I13','I15']
chf_10=['I099', 'I110', 'I130','I13.2', 'I25.5', 'I42.0', 'I425','I426','I427','I428','I429','I43', 'I50', 'P290']
cardiac_10=['I441','I442','I443','I456','I459', 'I47', 'I48', 'I49', 'ROOO', 'ROO1', 'ROO8', 'T821', 'Z450', 'Z950']
valvular_disease_10=['A520','I05','I06','I07','I08','I091','I098', 'I34','I35','I36','I37','I38','I39', 'Q23O','Q231','Q232','Q233', 'Z952', 'Z954']
Pulmonary_circulation_10=['I26', 'I27', 'I280', 'I288', 'I289']
Peripheral_vascular_10=['I70', 'I71', 'I731', 'I738', 'I739', 'I771', 'I790', 'I792', 'K551', 'K558', 'K559', 'Z958', 'Z959' ]
Paralysis_10=['G041', 'G114', 'G801', 'G802', 'G81', 'G82','G830','G831','G832','G833','G834', 'G839' ]
Other_neurological_disorders_10=['G10','G11','G12','G13', 'G20','G21','G22', 'G254', 'G255', 'G312', 'G318', 'G319', 'G32', 'G35','G36','G37', 'G40', 'G41', 'G931', 'G934', 'R470', 'R56']
Chronic_pulmonary_disease_10=['I278', 'I279','J40','J41','J42','J43','J44','J45','J46','J47','J60','J61','J62','J63','J64','J65','J66','J67','J684', 'J701', 'J703' ]
Diabetes_uncomplicated_10=['E100', 'E101', 'E109', 'E110', 'E111', 'E119', 'E120', 'E121', 'E129', 'E130', 'E131','E139', 'E140', 'E141', 'E149' ]
Diabetes_complicated_10=['E102','E103','E104','E105','E106','E107','E108', 'E112','E113','E114','E115','E116','E117','E118','E122','E123','E124','E125','E126','E127','E128','E132','E133','E134','E135','E136''E137','E138','E142','E143','E144','E145','E146','E147','E148']
Hypothyroidism_10=['E00','E01','E02','E03','E890']
Renal_failure_10=['I120', 'I131', 'N18', 'NI9', 'N250', 'Z490', 'Z491', 'Z492','Z940', 'Z992' ]
Liver_disease_10=['B18', 'I85', 'I864', 'I982', 'K70', 'K711', 'K713','K715', 'K717', 'K72', 'K73', 'K74', 'K76','K762','K763','K764','K765','K766','K767','K768','K769','Z944']
Pepticulcer_disease_10=['K257', 'K259', 'K267', 'K269','K277', 'K279', 'K287', 'K289']
AIDS_H1V_10=['B20','B21','B22','B24']
Lymphoma_10=['C81','C82','C83','C84','C85', 'C88','C96', 'C90', 'C902']
Metastatic_cancer_10=['C77','C78','C79','C80']
Solid_tumor_without_metastasis_10=['C00','C01','C02','C03','C04','C05','C06','C07','C08','C09','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26','C30','C31','C32','C33','C34','C37','C38','C39','C40','C41','C43','C45','C46','C47','C48','C49','C50','C51','C52','C53','C54','C55','C56','C57','C58','C60','C61','C62','C63','C64','C65','C66','C67','C68','C69','C70','C71','C72','C73','C74',  'C75', 'C76','C97']
Rheumatoid_arthritis_collagen_vascular_diseases_10=['L940', 'L941', 'L943', 'M05','M06', 'M08', 'M120','M123', 'M30', 'M310','M311','M312','M313','M32','M33','M34','M35','M45','M461', 'M468','M469']
Coagulopathy_10=['D65','D66','D67','D68','D691','D693','D694','D695','D696']
Obesity_10=['E66']
Weight_loss_10=['E40','E41','E42','E43','E44','E45','E46','R634', 'R64']
Fluid_electrolyte_disorders_10=['E222','E86','E87']
Blood_loss_anemia_10=['D500']
Deficiency_anemia_10=['D508', 'D509', 'D51','D52','D53']
Alcohol_abuse_10=['F10', 'E52', 'G621', 'I426', 'K292', 'K700', 'K703','K709', 'T51', 'Z502','Z714', 'Z721' ]
Drug_abuse_10=['F11','F12','F13','F14','F15','F16','F18','F19','Z715','Z722' ]
Psychoses_10=['F20','F22','F23','F24','F25','F28','F29', 'F302', 'F312', 'F315']
Depression_10=['F204','F313','F314','F315', 'F32', 'F33', 'F341', 'F412', 'F432' ]

In [55]:
combined_list_10 = (hypertension_uncomplicated_10 +hypertension_complicated_10 +chf_10 +cardiac_10 +valvular_disease_10 +Pulmonary_circulation_10 +Peripheral_vascular_10 +Paralysis_10 +Other_neurological_disorders_10 +Chronic_pulmonary_disease_10 +Diabetes_uncomplicated_10 +Diabetes_complicated_10 +Hypothyroidism_10 +Renal_failure_10 +Liver_disease_10 +Pepticulcer_disease_10 +AIDS_H1V_10 +Lymphoma_10 +Metastatic_cancer_10 +Solid_tumor_without_metastasis_10 +Rheumatoid_arthritis_collagen_vascular_diseases_10 +Coagulopathy_10 +Obesity_10 +Weight_loss_10 +Fluid_electrolyte_disorders_10 +Blood_loss_anemia_10 +Deficiency_anemia_10 +Alcohol_abuse_10 +Drug_abuse_10 +Psychoses_10 +Depression_10)

In [56]:
icd_9_mask = (diag_df['icd_code'].str.startswith(tuple(combined_list_9)))&(diag_df['icd_version']==9)
icd_10_mask = (diag_df['icd_code'].str.startswith(tuple(combined_list_10)))&(diag_df['icd_version']==10)
icd_9_rows = diag_df[icd_9_mask]
icd_10_rows = diag_df[icd_10_mask]

In [57]:
diag_df_1=pd.concat([icd_9_rows,icd_10_rows])

In [58]:
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(hypertension_uncomplicated_9)))&(diag_df['icd_version']==9), 'name'] = 'hypertension_uncomplicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(hypertension_complicated_9)))&(diag_df['icd_version']==9), 'name'] = 'hypertension_complicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(chf_9)))&(diag_df['icd_version']==9), 'name'] = 'chf'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(cardiac_9)))&(diag_df['icd_version']==9), 'name'] = 'cardiac'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(valvular_disease_9)))&(diag_df['icd_version']==9), 'name'] = 'valvular_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Pulmonary_circulation_9)))&(diag_df['icd_version']==9), 'name'] = 'Pulmonary_circulation'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Peripheral_vascular_9)))&(diag_df['icd_version']==9), 'name'] = 'Peripheral_vascular'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Paralysis_9)))&(diag_df['icd_version']==9), 'name'] = 'Paralysis'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Other_neurological_disorders_9)))&(diag_df['icd_version']==9), 'name'] = 'Other_neurological_disorders'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Chronic_pulmonary_disease_9)))&(diag_df['icd_version']==9), 'name'] = 'Chronic_pulmonary_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Diabetes_uncomplicated_9)))&(diag_df['icd_version']==9), 'name'] = 'Diabetes_uncomplicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Diabetes_complicated_9)))&(diag_df['icd_version']==9), 'name'] = 'Diabetes_complicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Hypothyroidism_9)))&(diag_df['icd_version']==9), 'name'] = 'Hypothyroidism'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Renal_failure_9)))&(diag_df['icd_version']==9), 'name'] = 'Renal_failure'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Liver_disease_9)))&(diag_df['icd_version']==9), 'name'] = 'Liver_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Pepticulcer_disease_9)))&(diag_df['icd_version']==9), 'name'] = 'Pepticulcer_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(AIDS_H1V_9)))&(diag_df['icd_version']==9), 'name'] = 'AIDS_H1V'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Lymphoma_9)))&(diag_df['icd_version']==9), 'name'] = 'Lymphoma'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Metastatic_cancer_9)))&(diag_df['icd_version']==9), 'name'] = 'Metastatic_cancer'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Solid_tumor_without_metastasis_9)))&(diag_df['icd_version']==9), 'name'] = 'Solid_tumor_without_metastasis'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Rheumatoid_arthritis_collagen_vascular_diseases_9)))&(diag_df['icd_version']==9), 'name'] = 'Rheumatoid_arthritis_collagen_vascular_diseases'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Coagulopathy_9)))&(diag_df['icd_version']==9), 'name'] = 'Coagulopathy'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Obesity_9)))&(diag_df['icd_version']==9), 'name'] = 'Obesity'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Weight_loss_9)))&(diag_df['icd_version']==9), 'name'] = 'Weight_loss'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Fluid_electrolyte_disorders_9)))&(diag_df['icd_version']==9), 'name'] = 'Fluid_electrolyte_disorders'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Blood_loss_anemia_9)))&(diag_df['icd_version']==9), 'name'] = 'Blood_loss_anemia'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Deficiency_anemia_9)))&(diag_df['icd_version']==9), 'name'] = 'Deficiency_anemia'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Alcohol_abuse_9)))&(diag_df['icd_version']==9), 'name'] = 'Alcohol_abuse'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Drug_abuse_9)))&(diag_df['icd_version']==9), 'name'] = 'Drug_abuse'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Psychoses_9)))&(diag_df['icd_version']==9), 'name'] = 'Psychoses'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Depression_9)))&(diag_df['icd_version']==9), 'name'] = 'Depression'

In [59]:
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(hypertension_uncomplicated_10)))&(diag_df['icd_version']== 10), 'name'] = 'hypertension_uncomplicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(hypertension_complicated_10)))&(diag_df['icd_version']== 10), 'name'] = 'hypertension_complicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(chf_10)))&(diag_df['icd_version']== 10), 'name'] = 'chf'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(cardiac_10)))&(diag_df['icd_version']== 10), 'name'] = 'cardiac'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(valvular_disease_10)))&(diag_df['icd_version']== 10), 'name'] = 'valvular_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Pulmonary_circulation_10)))&(diag_df['icd_version']== 10), 'name'] = 'Pulmonary_circulation'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Peripheral_vascular_10)))&(diag_df['icd_version']== 10), 'name'] = 'Peripheral_vascular'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Paralysis_10)))&(diag_df['icd_version']== 10), 'name'] = 'Paralysis'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Other_neurological_disorders_10)))&(diag_df['icd_version']== 10), 'name'] = 'Other_neurological_disorders'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Chronic_pulmonary_disease_10)))&(diag_df['icd_version']== 10), 'name'] = 'Chronic_pulmonary_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Diabetes_uncomplicated_10)))&(diag_df['icd_version']== 10), 'name'] = 'Diabetes_uncomplicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Diabetes_complicated_10)))&(diag_df['icd_version']== 10), 'name'] = 'Diabetes_complicated'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Hypothyroidism_10)))&(diag_df['icd_version']== 10), 'name'] = 'Hypothyroidism'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Renal_failure_10)))&(diag_df['icd_version']== 10), 'name'] = 'Renal_failure'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Liver_disease_10)))&(diag_df['icd_version']== 10), 'name'] = 'Liver_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Pepticulcer_disease_10)))&(diag_df['icd_version']== 10), 'name'] = 'Pepticulcer_disease'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(AIDS_H1V_10)))&(diag_df['icd_version']== 10), 'name'] = 'AIDS_H1V'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Lymphoma_10)))&(diag_df['icd_version']== 10), 'name'] = 'Lymphoma'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Metastatic_cancer_10)))&(diag_df['icd_version']== 10), 'name'] = 'Metastatic_cancer'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Solid_tumor_without_metastasis_10)))&(diag_df['icd_version']== 10), 'name'] = 'Solid_tumor_without_metastasis'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Rheumatoid_arthritis_collagen_vascular_diseases_10)))&(diag_df['icd_version']== 10), 'name'] = 'Rheumatoid_arthritis_collagen_vascular_diseases'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Coagulopathy_10)))&(diag_df['icd_version']== 10), 'name'] = 'Coagulopathy'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Obesity_10)))&(diag_df['icd_version']== 10), 'name'] = 'Obesity'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Weight_loss_10)))&(diag_df['icd_version']== 10), 'name'] = 'Weight_loss'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Fluid_electrolyte_disorders_10)))&(diag_df['icd_version']== 10), 'name'] = 'Fluid_electrolyte_disorders'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Blood_loss_anemia_10)))&(diag_df['icd_version']== 10), 'name'] = 'Blood_loss_anemia'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Deficiency_anemia_10)))&(diag_df['icd_version']== 10), 'name'] = 'Deficiency_anemia'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Alcohol_abuse_10)))&(diag_df['icd_version']== 10), 'name'] = 'Alcohol_abuse'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Drug_abuse_10)))&(diag_df['icd_version']== 10), 'name'] = 'Drug_abuse'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Psychoses_10)))&(diag_df['icd_version']== 10), 'name'] = 'Psychoses'
diag_df_1.loc[(diag_df_1['icd_code'].str.startswith(tuple(Depression_10)))&(diag_df['icd_version']== 10), 'name'] = 'Depression'

In [60]:
diag_df_1
#1462704 

,hadm_id,icd_code,icd_version,name
0,22595853,5723,9,Liver_disease
2,22595853,5715,9,Liver_disease
4,22595853,496,9,Chronic_pulmonary_disease
6,22595853,30981,9,Depression
10,22841357,2875,9,Coagulopathy
...,...,...,...,...
4756281,29734428,I480,10,cardiac
4756282,29734428,I10,10,hypertension_uncomplicated
4756283,29734428,J449,10,Chronic_pulmonary_disease
4756284,29734428,E119,10,Diabetes_uncomplicated


In [61]:
diag_df_2=diag_df_1[diag_df_1['hadm_id'].isin(ward['hadm_id'])]
diag_df_2=diag_df_2[['hadm_id','name']]

In [62]:
diag_df_2

,hadm_id,name
252,24947999,chf
253,24947999,Renal_failure
254,24947999,Diabetes_uncomplicated
256,24947999,Obesity
258,24947999,hypertension_complicated
...,...,...
4755301,25785472,Fluid_electrolyte_disorders
4755302,25785472,chf
4755303,25785472,chf
4755311,25785472,Depression


In [63]:
diag_df_2.rename(columns={'name': 'Comorbidity'}, inplace=True)
one_hot_diag_df_2 = pd.get_dummies(diag_df_2, columns=['Comorbidity']).astype(int)

In [64]:
one_hot_diag_df_2=one_hot_diag_df_2.sort_values(by='hadm_id')

In [65]:
diag_df_4=one_hot_diag_df_2.groupby('hadm_id').max().reset_index()
diag_df_4
#46887

,hadm_id,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,20000019,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,20000024,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,20000343,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
3,20000750,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,20001002,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46882,29998469,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
46883,29998779,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
46884,29999161,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
46885,29999501,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [66]:
hos_data=pd.merge(ward, diag_df_4,on='hadm_id',how='left')

In [67]:
hos_data

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease
0,20000019.0,2159-03-20 22:48:00,Other,F,76,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,20000019.0,2159-03-23 16:54:19,Other,F,76,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20000024.0,2151-05-26 00:03:00,White,F,92,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,20000024.0,2151-05-26 18:36:48,White,F,92,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,20000343.0,2137-01-28 01:24:00,Black,M,59,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100525,29999161.0,2175-03-21 12:43:43,White,M,63,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
100526,29999501.0,2135-06-26 17:28:00,White,F,76,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
100527,29999501.0,2135-06-29 14:46:10,White,F,76,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
100528,29999723.0,2170-08-09 19:50:00,White,M,61,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [68]:
dfdf_1=dfdf[['hadm_id','intime_admit','outtime']]
hos_data = pd.merge(hos_data,dfdf_1, how='left' , on='hadm_id')

In [69]:
hos_data
hos_data.rename(columns = {'intime_admit' : 'intime'}, inplace = True)

In [70]:
hos_data

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime
0,20000019.0,2159-03-20 22:48:00,Other,F,76,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19
1,20000019.0,2159-03-23 16:54:19,Other,F,76,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19
2,20000024.0,2151-05-26 00:03:00,White,F,92,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48
3,20000024.0,2151-05-26 18:36:48,White,F,92,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48
4,20000343.0,2137-01-28 01:24:00,Black,M,59,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2137-01-28 01:24:00,2137-01-30 18:00:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100525,29999161.0,2175-03-21 12:43:43,White,M,63,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2175-03-20 13:54:12,2175-03-21 12:43:43
100526,29999501.0,2135-06-26 17:28:00,White,F,76,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10
100527,29999501.0,2135-06-29 14:46:10,White,F,76,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10
100528,29999723.0,2170-08-09 19:50:00,White,M,61,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2170-08-09 19:50:00,2170-08-10 18:34:46


In [172]:
service=pd.read_parquet('services.parquet')
service

,subject_id,hadm_id,transfertime,prev_service,curr_service
0,10000032,22595853,2180-05-06 22:24:57,None,MED
1,10000032,22841357,2180-06-26 18:28:08,None,MED
2,10000032,25742920,2180-08-05 23:44:50,None,MED
3,10000032,29079034,2180-07-23 12:36:04,None,MED
4,10000068,25022803,2160-03-03 23:17:17,None,MED
...,...,...,...,...,...
468024,19999828,29734428,2147-07-18 16:24:15,None,SURG
468025,19999840,21033226,2164-09-10 13:49:01,None,NMED
468026,19999840,21033226,2164-09-14 10:40:18,NMED,MED
468027,19999840,26071774,2164-07-25 00:27:59,None,NMED


In [173]:
service['stoptime'] = service.groupby('hadm_id')['transfertime'].shift(-1)
service

,subject_id,hadm_id,transfertime,prev_service,curr_service,stoptime
0,10000032,22595853,2180-05-06 22:24:57,None,MED,NaN
1,10000032,22841357,2180-06-26 18:28:08,None,MED,NaN
2,10000032,25742920,2180-08-05 23:44:50,None,MED,NaN
3,10000032,29079034,2180-07-23 12:36:04,None,MED,NaN
4,10000068,25022803,2160-03-03 23:17:17,None,MED,NaN
...,...,...,...,...,...,...
468024,19999828,29734428,2147-07-18 16:24:15,None,SURG,NaN
468025,19999840,21033226,2164-09-10 13:49:01,None,NMED,2164-09-14 10:40:18
468026,19999840,21033226,2164-09-14 10:40:18,NMED,MED,NaN
468027,19999840,26071774,2164-07-25 00:27:59,None,NMED,NaN


In [174]:
service = service[['hadm_id','transfertime','stoptime','curr_service']]
service_1 = service[service['hadm_id'].isin(hos_data['hadm_id'])].reset_index(drop=True)

In [175]:
service_1['transfertime'] = pd.to_datetime(service_1['transfertime'])
service_1.rename(columns={'transfertime': 'time'}, inplace=True)

In [176]:
service_1['curr_service'].value_counts()

curr_service
MED      44360
SURG      2526
OMED      1175
NMED       916
ORTHO      697
VSURG      532
CMED       531
TRAUM      219
PSURG      202
GU          89
TSURG       79
GYN         78
NSURG       70
ENT         46
PSYCH       26
CSURG       22
OBS         14
EYE          4
DENT         1
Name: count, dtype: int64

In [177]:
service_1['hadm_id'].nunique()
#50265

50265

In [179]:
hos_data

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime
0,20000019.0,2159-03-20 22:48:00,Other,F,76,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19
1,20000019.0,2159-03-23 16:54:19,Other,F,76,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19
2,20000024.0,2151-05-26 00:03:00,White,F,92,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48
3,20000024.0,2151-05-26 18:36:48,White,F,92,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48
4,20000343.0,2137-01-28 01:24:00,Black,M,59,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2137-01-28 01:24:00,2137-01-30 18:00:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100525,29999161.0,2175-03-21 12:43:43,White,M,63,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2175-03-20 13:54:12,2175-03-21 12:43:43
100526,29999501.0,2135-06-26 17:28:00,White,F,76,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10
100527,29999501.0,2135-06-29 14:46:10,White,F,76,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10
100528,29999723.0,2170-08-09 19:50:00,White,M,61,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2170-08-09 19:50:00,2170-08-10 18:34:46


In [178]:
hos_data_1=pd.concat([hos_data,service_1])
hos_data_1=hos_data_1.sort_values(by=['hadm_id','time'])
hos_data_1.reset_index(drop=True,inplace=True)
hos_data_1

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service
0,20000019.0,2159-03-20 21:10:17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,MED
1,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,NaN
2,20000019.0,2159-03-23 16:54:19,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,NaN
3,20000024.0,2151-05-25 21:45:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,MED
4,20000024.0,2151-05-26 00:03:00,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152112,29999501.0,2135-06-26 17:28:00,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,NaN,NaN
152113,29999501.0,2135-06-29 14:46:10,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,NaN,NaN
152114,29999723.0,2170-08-09 18:57:07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,MED
152115,29999723.0,2170-08-09 19:50:00,White,M,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2170-08-09 19:50:00,2170-08-10 18:34:46,NaN,NaN


In [180]:
hos_data_1['intime']=hos_data_1.groupby('hadm_id')['intime'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
hos_data_1['outtime']=hos_data_1.groupby('hadm_id')['outtime'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

C:\Users\DAHS\AppData\Local\Temp\ipykernel_2220\1235203020.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hos_data_1['intime']=hos_data_1.groupby('hadm_id')['intime'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\DAHS\AppData\Local\Temp\ipykernel_2220\1235203020.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hos_data_1['outtime']=hos_data_1.groupby('hadm_id')['outtime'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))


In [181]:
hos_data_2 = hos_data_1[
    (hos_data_1['time'] >= hos_data_1['intime']) &
    (hos_data_1['time'] <= hos_data_1['outtime'])]

### intime 기준이 med를 입원 한 후를 측정해서 그런가 service는 med 오기전에 대부분이 찍힌다. 

In [182]:
hos_data_2=hos_data_2.sort_values(by=['hadm_id','time'])
hos_data_2.reset_index(drop=True,inplace=True)
hos_data_2

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,NaN
1,20000019.0,2159-03-23 16:54:19,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,NaN
2,20000024.0,2151-05-26 00:03:00,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,NaN,NaN
3,20000024.0,2151-05-26 18:36:48,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,NaN,NaN
4,20000343.0,2137-01-28 01:24:00,Black,M,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2137-01-28 01:24:00,2137-01-30 18:00:08,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111108,29999161.0,2175-03-21 12:43:43,White,M,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2175-03-20 13:54:12,2175-03-21 12:43:43,NaN,NaN
111109,29999501.0,2135-06-26 17:28:00,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,NaN,NaN
111110,29999501.0,2135-06-29 14:46:10,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,NaN,NaN
111111,29999723.0,2170-08-09 19:50:00,White,M,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2170-08-09 19:50:00,2170-08-10 18:34:46,NaN,NaN


In [183]:
import pandas as pd

dummies = pd.get_dummies(hos_data_2['curr_service'], prefix='curr_service')

dummies = dummies.astype(int)

hos_data_2 = hos_data_2.drop('curr_service', axis=1)
hos_data_3 = pd.concat([hos_data_2, dummies], axis=1)

In [184]:
dummies

,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111108,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111109,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111110,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111111,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [185]:
hos_data_3

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20000019.0,2159-03-23 16:54:19,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,20000024.0,2151-05-26 00:03:00,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,20000024.0,2151-05-26 18:36:48,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20000343.0,2137-01-28 01:24:00,Black,M,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2137-01-28 01:24:00,2137-01-30 18:00:08,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111108,29999161.0,2175-03-21 12:43:43,White,M,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2175-03-20 13:54:12,2175-03-21 12:43:43,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111109,29999501.0,2135-06-26 17:28:00,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111110,29999501.0,2135-06-29 14:46:10,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111111,29999723.0,2170-08-09 19:50:00,White,M,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2170-08-09 19:50:00,2170-08-10 18:34:46,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [186]:
hos_data_3['hadm_id'].value_counts()

hadm_id
21860318.0    7
24850803.0    6
24708605.0    6
26961252.0    6
26510562.0    5
             ..
23692113.0    2
23692334.0    2
23692416.0    2
23692472.0    2
29999723.0    2
Name: count, Length: 50265, dtype: int64

### suspected infection

In [84]:
blood_culture = [225401, 225437, 'BLOOD CULTURE', 'BLOOD CULTURE ( MYCO/F LYTIC BOTTLE)', 'Stem Cell - Blood Culture', 'BLOOD CULTURE (POST-MORTEM)', 'POSTMORTEM CULTURE', 'BLOOD CULTURE - NEONATE']

antibiotic_name = ['amikacin', 'gentamicin', 'kanamycin', 'netilmicin', 'tobramycin', 'paromomycin', 'spectinomycin', 'geldanamycin','ertapenem', 'doripenem', 'imipenem', 'meropenem', 'cefadroxil', 'cefalexin', 'cefaclor', 'cefoxitin', 'cefprozil','cefamandole', 'cefuroxime', 'cefixime', 'cefotaxime', 'cefpodoxime', 'ceftazidime', 'ceftriaxone', 'cefepime','vancomycin', 'vanc', 'clindamycin', 'daptomycin', 'azithromycin', 'clarithromycin', 'erythromycin', 'telithromycin','aztreonam', 'nitrofurantoin', 'linezolid', 'amoxicillin', 'ampicillin', 'dicloxacillin', 'flucloxacillin','methicillin', 'nafcillin', 'oxacillin', 'penicillin', 'piperacillin', 'cefotetan', 'ticarcillin', 'timentin', 'colistin', 'bactrim', 'polymyxin', 'ciprofloxacin', 'gatifloxacin', 'levofloxacin', 'moxifloxacin', 'nalidixic acid','norfloxacin', 'ofloxacin', 'trovafloxacin', 'sulfadiazine', 'sulfamethoxazole', 'trimethoprim', 'TMP', 'doxycycline', 'minocycline','tetracycline', 'dapsone', 'ethambutol', 'isoniazid', 'pyrazinamide', 'rifampicin', 'rifampin', 'rifabutin', 'streptomycin','chloramphenicol', 'synercid', 'fosfomycin', 'metronidazole', 'mupirocin', 'quinupristin', 'tigecycline', 'unasyn']

In [85]:
sus_icd = pd.read_excel('suspected_infection.xlsx')

In [86]:
sus_icd_9 = sus_icd[sus_icd['icd_version']==9]
sus_icd_10 = sus_icd[sus_icd['icd_version']==10]

In [87]:
sus_icd_9 = ['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035','036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '320', '321', '322', '323', '324', '325', '326', '42090', '420', '421', '422', '4476', '460', '461', '462', '463', '464', '465', '482', '483', '484', '485', '486', '487', '488', '466', '523', '52800', '5283', '6820', '53110', '53120', '53160', '53210', '53220', '53250', '53410', '53420', '53450', '542', '541', '562', '566', '5695', '56983', '567', '99859', '572', '5740', '5741', '5742', '5743', '5744', '5745', '5746', '5747', '5748', '5750', '5751', '5754', '5759', '5761', '5763', '681', '682', '683', '684', '685', '686', '711', '590', '595', '597', '5990', '601', '604', '608', '611', '614', '615', '616', '646', '670', '647', '675', '672', '7806', '7806', '78559', '7855', '99592', '78552', '7919', '7920', '7929', '7924', '7922', '7929', '7907']

In [88]:
sus_icd_10 = ['A', 'B', 'G00', 'G01', 'G02', 'G03', 'G04', 'G05', 'G06', 'G07', 'G08', 'G09', 'I30', 'I32', 'I33', 'I40', 'I776', 'J00', 'J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J09', 'J10', 'J11', 'J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J20', 'J21', 'J22', 'K05', 'K122', 'K251', 'K252', 'K256', 'K261', 'K262', 'K265', 'K281', 'K282', 'K285', 'K35', 'K36', 'K37', 'K57', 'K61', 'K630', 'K631', 'K65', 'K681', 'K750', 'K800', 'K801', 'K803', 'K804', 'K806', 'K81', 'K822', 'K82A', 'K830', 'K832', 'L00', 'L01', 'L02', 'L03', 'L04', 'L05', 'L06', 'L07', 'L08', 'M00', 'M01', 'M02', 'N10', 'N30', 'N34', 'N390', 'N41', 'N45', 'N49', 'N61', 'N70', 'N71', 'N72', 'N73', 'N74', 'N75', 'N76', 'N77', 'O23', 'O85', 'O86', 'O91', 'O98', 'R508', 'R509', 'R578', 'R579', 'R652', 'R827', 'R835', 'R845', 'R855', 'R865', 'R875', 'R7881']

In [89]:
version_9_df = diag_df[diag_df['icd_version'] == 9]

# sus_icd에서 icd_code를 리스트로 추출
matching_hadm_ids_9 = version_9_df[version_9_df['icd_code'].apply(lambda code: any(code.startswith(icd) for icd in sus_icd_9))]


In [90]:
version_10_df = diag_df[diag_df['icd_version'] == 10]

# sus_icd에서 icd_code를 리스트로 추출
matching_hadm_ids_10 = version_10_df[version_10_df['icd_code'].apply(lambda code: any(code.startswith(icd) for icd in sus_icd_10))]


In [91]:
matching_hadm_ids_10

,hadm_id,icd_code,icd_version
314,25911675,K5730,10
461,24818636,N390,10
466,24818636,B952,10
483,26840593,K651,10
488,26840593,B966,10
...,...,...,...
4756241,29889147,B1910,10
4756259,25744818,L02211,10
4756271,25744818,B954,10
4756278,29734428,N390,10


In [92]:
con_1 = set(matching_hadm_ids_10['hadm_id'])|set(matching_hadm_ids_9['hadm_id'])

In [93]:
len(con_1)

155688

In [94]:
micro=pd.read_parquet('microbiologyevents.parquet')
micro

,microevent_id,subject_id,hadm_id,micro_specimen_id,order_provider_id,chartdate,charttime,spec_itemid,spec_type_desc,test_seq,storedate,storetime,test_itemid,test_name,org_itemid,org_name,isolate_num,quantity,ab_itemid,ab_name,dilution_text,dilution_comparison,dilution_value,interpretation,comments
0,1,10000032,NaN,636109,P28Z0X,2180-03-23 00:00:00,2180-03-23 11:51:00,70093,Blood (Toxo),1,2180-03-26 00:00:00,2180-03-26 10:17:00,90144,TOXOPLASMA IgG ANTIBODY,NaN,None,NaN,None,NaN,None,None,None,NaN,None,NEGATIVE FOR TOXOPLASMA IgG ANTIBODY BY EIA. ...
1,2,10000032,NaN,1836584,P28Z0X,2180-03-23 00:00:00,2180-03-23 11:51:00,70017,SEROLOGY/BLOOD,1,2180-03-24 00:00:00,2180-03-24 12:40:00,90127,"RUBEOLA ANTIBODY, IgG",NaN,None,NaN,None,NaN,None,None,None,NaN,None,POSITIVE BY EIA. A positive IgG result genera...
2,3,10000032,NaN,4131591,P28Z0X,2180-03-23 00:00:00,2180-03-23 11:51:00,70087,Blood (CMV AB),1,2180-03-26 00:00:00,2180-03-26 10:06:00,90143,CMV IgG ANTIBODY,NaN,None,NaN,None,NaN,None,None,None,NaN,None,___
3,4,10000032,NaN,4131591,P28Z0X,2180-03-23 00:00:00,2180-03-23 11:51:00,70087,Blood (CMV AB),2,2180-03-26 00:00:00,2180-03-26 10:06:00,90136,CMV IgM ANTIBODY,NaN,None,NaN,None,NaN,None,None,None,NaN,None,NEGATIVE FOR CMV IgM ANTIBODY BY EIA. INTERPR...
4,5,10000032,NaN,6028147,P28Z0X,2180-03-23 00:00:00,2180-03-23 11:51:00,70088,Blood (EBV),1,2180-03-25 00:00:00,2180-03-25 11:54:00,90101,EPSTEIN-BARR VIRUS VCA-IgG AB,NaN,None,NaN,None,NaN,None,None,None,NaN,None,POSITIVE BY EIA.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228708,3228709,19999987,23865745.0,9008390,None,2145-11-05 00:00:00,2145-11-05 10:35:00,70012,BLOOD CULTURE,1,2145-11-11 00:00:00,2145-11-11 07:39:00,90201,"Blood Culture, Routine",NaN,None,NaN,None,NaN,None,None,None,NaN,None,NO GROWTH.
3228709,3228710,19999987,23865745.0,3162785,None,2145-11-05 00:00:00,2145-11-05 12:15:00,70012,BLOOD CULTURE,1,2145-11-11 00:00:00,2145-11-11 07:40:00,90201,"Blood Culture, Routine",NaN,None,NaN,None,NaN,None,None,None,NaN,None,NO GROWTH.
3228710,3228711,19999987,23865745.0,5530834,None,2145-11-06 00:00:00,2145-11-06 13:15:00,70012,BLOOD CULTURE,1,2145-11-12 00:00:00,2145-11-12 07:31:00,90201,"Blood Culture, Routine",NaN,None,NaN,None,NaN,None,None,None,NaN,None,NO GROWTH.
3228711,3228712,19999987,23865745.0,8701158,None,2145-11-06 00:00:00,2145-11-06 17:05:00,70012,BLOOD CULTURE,1,2145-11-12 00:00:00,2145-11-12 07:33:00,90201,"Blood Culture, Routine",NaN,None,NaN,None,NaN,None,None,None,NaN,None,NO GROWTH.


In [96]:
con_2=set(micro[micro['spec_type_desc'].isin(blood_culture)]['hadm_id'].unique())

In [97]:
len(con_2)

71884

In [98]:
anti=pd.read_parquet('pharmacy.parquet')
anti

,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,proc_type,status,entertime,verifiedtime,route,frequency,disp_sched,infusion_type,sliding_scale,lockout_interval,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
0,10000032,22595853,11700683,10000032-34,2180-05-07 01:00:00,2180-05-07 22:00:00,Acetaminophen,Unit Dose,Discontinued via patient discharge,2180-05-07 00:09:24,2180-05-07 00:09:24,PO/NG,Q6H:PRN,None,None,None,None,NaN,None,NaN,NaN,Ongoing,36.0,Hours,None,Omnicell,None
1,10000032,22595853,14779570,10000032-22,2180-05-07 00:00:00,2180-05-07 22:00:00,Sodium Chloride 0.9% Flush,Unit Dose,Discontinued via patient discharge,2180-05-07 00:00:54,2180-05-07 00:00:54,IV,Q8H,"0, 8, 16",None,None,None,NaN,None,3.0,NaN,Ongoing,36.0,Hours,None,Floor Stock Item,None
2,10000032,22595853,19796602,10000032-50,2180-05-08 08:00:00,2180-05-07 22:00:00,Furosemide,Unit Dose,Discontinued via patient discharge,2180-05-07 09:32:35,2180-05-07 09:32:35,PO/NG,DAILY,08,None,None,None,NaN,None,1.0,NaN,Ongoing,36.0,Hours,None,Omnicell,None
3,10000032,22595853,20256254,10000032-32,2180-05-07 01:00:00,2180-05-07 22:00:00,Raltegravir,Unit Dose,Discontinued via patient discharge,2180-05-07 00:09:24,2180-05-07 00:09:24,PO,BID,"08, 20",None,None,None,NaN,None,2.0,NaN,Ongoing,36.0,Hours,None,Omnicell,None
4,10000032,22595853,28781051,10000032-27,2180-05-07 00:00:00,2180-05-07 22:00:00,Heparin,Unit Dose,Discontinued via patient discharge,2180-05-07 00:00:54,2180-05-07 00:00:54,SC,TID,"08, 14, 20",None,None,None,NaN,None,3.0,NaN,Ongoing,36.0,Hours,None,Omnicell,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13584509,19999987,23865745,92839339,19999987-25,2145-11-03 00:00:00,2145-11-03 18:00:00,Influenza Virus Vaccine,Unit Dose,Discontinued,2145-11-02 23:06:13,2145-11-02 23:06:13,IM,NOW X1,None,None,None,None,NaN,None,0.0,NaN,Ongoing,36.0,Hours,None,Omnicell,None
13584510,19999987,23865745,92913309,19999987-150,2145-11-04 10:00:00,2145-11-11 17:00:00,Levothyroxine Sodium,Unit Dose,Discontinued via patient discharge,2145-11-04 22:39:21,2145-11-04 22:39:21,PO/NG,DAILY,10,None,None,None,NaN,None,1.0,NaN,Ongoing,36.0,Hours,None,Omnicell,None
13584511,19999987,23865745,95753195,19999987-219,2145-11-10 10:00:00,2145-11-11 17:00:00,Duloxetine,Unit Dose,Discontinued via patient discharge,2145-11-10 10:59:47,2145-11-10 10:59:47,PO,DAILY,10,None,None,None,NaN,None,1.0,NaN,Ongoing,36.0,Hours,None,Omnicell,None
13584512,19999987,23865745,96343043,19999987-206,2145-11-09 10:00:00,2145-11-09 16:00:00,Venlafaxine XR,Unit Dose,Discontinued,2145-11-09 15:35:14,2145-11-09 15:35:14,PO,DAILY,10,None,None,None,NaN,None,1.0,NaN,Ongoing,36.0,Hours,None,Omnicell,None


In [99]:
#anti 단순화 
anti['medication'] = anti['medication'].str.lower()
anti['medication']= anti['medication'].str.replace(" ", "")

In [105]:
con_3 = set(anti[anti['medication'].isin(antibiotic_name)]['hadm_id'].unique())

In [108]:
sus_id = con_1&con_2&con_3

In [187]:
hos_data_3

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20000019.0,2159-03-23 16:54:19,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,20000024.0,2151-05-26 00:03:00,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,20000024.0,2151-05-26 18:36:48,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20000343.0,2137-01-28 01:24:00,Black,M,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2137-01-28 01:24:00,2137-01-30 18:00:08,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111108,29999161.0,2175-03-21 12:43:43,White,M,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2175-03-20 13:54:12,2175-03-21 12:43:43,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111109,29999501.0,2135-06-26 17:28:00,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111110,29999501.0,2135-06-29 14:46:10,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111111,29999723.0,2170-08-09 19:50:00,White,M,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2170-08-09 19:50:00,2170-08-10 18:34:46,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [188]:
hos_data_3['sus_infec'] = 0

# hadm_id가 sus_id 리스트에 포함되어 있는 경우, 'sus_infec'를 1로 설정
hos_data_3.loc[hos_data_3['hadm_id'].isin(sus_id), 'sus_infec'] = 1

In [189]:
hos_data_3

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,20000019.0,2159-03-23 16:54:19,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,20000024.0,2151-05-26 00:03:00,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,20000024.0,2151-05-26 18:36:48,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20000343.0,2137-01-28 01:24:00,Black,M,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2137-01-28 01:24:00,2137-01-30 18:00:08,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111108,29999161.0,2175-03-21 12:43:43,White,M,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2175-03-20 13:54:12,2175-03-21 12:43:43,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111109,29999501.0,2135-06-26 17:28:00,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111110,29999501.0,2135-06-29 14:46:10,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111111,29999723.0,2170-08-09 19:50:00,White,M,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2170-08-09 19:50:00,2170-08-10 18:34:46,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [191]:
#hos_data_3.to_csv('pre_hos_v0.2.csv',index=0)

In [116]:
hos_data_3

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,20000019.0,2159-03-23 16:54:19,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,20000024.0,2151-05-26 00:03:00,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,20000024.0,2151-05-26 18:36:48,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20000343.0,2137-01-28 01:24:00,Black,M,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2137-01-28 01:24:00,2137-01-30 18:00:08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111108,29999161.0,2175-03-21 12:43:43,White,M,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2175-03-20 13:54:12,2175-03-21 12:43:43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111109,29999501.0,2135-06-26 17:28:00,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111110,29999501.0,2135-06-29 14:46:10,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111111,29999723.0,2170-08-09 19:50:00,White,M,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2170-08-09 19:50:00,2170-08-10 18:34:46,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [192]:
drug_name = anti[anti['hadm_id'].isin(hos_data_3['hadm_id'])]['medication'].value_counts().head(20).reset_index()['medication'].to_list()

In [193]:
drug = anti[anti['hadm_id'].isin(hos_data_3['hadm_id'])]
using_drug = drug[drug['medication'].isin(drug_name)][['hadm_id','starttime','stoptime','medication']]
using_drug

,hadm_id,starttime,stoptime,medication
80,23052089,2160-11-21 04:00:00,2160-11-25 19:00:00,heparin
85,23052089,2160-11-21 04:00:00,2160-11-25 19:00:00,senna
87,23052089,2160-11-21 04:00:00,2160-11-25 19:00:00,senna
90,23052089,2160-11-21 04:00:00,2160-11-22 09:00:00,sodiumchloride0.9%flush
462,20897796,2193-08-15 10:00:00,2193-08-16 09:00:00,furosemide
...,...,...,...,...
13582285,24639135,2165-10-06 23:00:00,2165-10-07 22:00:00,hydromorphone(dilaudid)
13583526,27638769,2138-10-07 15:00:00,2138-10-09 21:00:00,senna
13583528,27638769,2138-10-07 15:00:00,2138-10-09 21:00:00,bisacodyl
13583532,27638769,2138-10-06 20:00:00,2138-10-09 21:00:00,heparin


In [194]:
using_drug['starttime']=pd.to_datetime(using_drug['starttime'])

using_drug['stoptime']=pd.to_datetime(using_drug['stoptime'])


In [197]:
using_drug_1=using_drug[using_drug['stoptime']>=using_drug['starttime']].reset_index(drop=True)

In [198]:
drug_diff = using_drug_1['stoptime']-using_drug_1['starttime']
drug_diff

0        4 days 15:00:00
1        4 days 15:00:00
2        4 days 15:00:00
3        1 days 05:00:00
4        0 days 23:00:00
               ...      
595437   0 days 23:00:00
595438   2 days 06:00:00
595439   2 days 06:00:00
595440   3 days 01:00:00
595441   2 days 06:00:00
Length: 595442, dtype: timedelta64[ns]

In [203]:
using_drug_1
using_drug_1.rename(columns={'starttime': 'time'}, inplace=True)

In [204]:
hos_data_3

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,20000019.0,2159-03-23 16:54:19,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,20000024.0,2151-05-26 00:03:00,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,20000024.0,2151-05-26 18:36:48,White,F,92.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2151-05-26 00:03:00,2151-05-26 18:36:48,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20000343.0,2137-01-28 01:24:00,Black,M,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2137-01-28 01:24:00,2137-01-30 18:00:08,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111108,29999161.0,2175-03-21 12:43:43,White,M,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2175-03-20 13:54:12,2175-03-21 12:43:43,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111109,29999501.0,2135-06-26 17:28:00,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111110,29999501.0,2135-06-29 14:46:10,White,F,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2135-06-26 17:28:00,2135-06-29 14:46:10,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111111,29999723.0,2170-08-09 19:50:00,White,M,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2170-08-09 19:50:00,2170-08-10 18:34:46,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [205]:
hos_data_4=pd.concat([hos_data_3,using_drug_1])
hos_data_4=hos_data_4.sort_values(by=['hadm_id','time'])
hos_data_4.reset_index(drop=True,inplace=True)
hos_data_4

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec,medication
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
1,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,2159-03-21 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,acetaminophen
2,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sodiumchloride0.9%flush
3,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,insulin
4,20000019.0,2159-03-21 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,docusatesodium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706550,29999723.0,2170-08-09 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,2170-08-10 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,heparin
706551,29999723.0,2170-08-09 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,2170-08-10 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,docusatesodium
706552,29999723.0,2170-08-09 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,2170-08-10 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sodiumchloride0.9%flush
706553,2999

In [206]:
hos_data_4['stoptime'] = pd.to_datetime(hos_data_4['stoptime'])

In [207]:
hos_data_4['intime']=hos_data_4.groupby('hadm_id')['intime'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
hos_data_4['outtime']=hos_data_4.groupby('hadm_id')['outtime'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

C:\Users\DAHS\AppData\Local\Temp\ipykernel_2220\972080183.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hos_data_4['intime']=hos_data_4.groupby('hadm_id')['intime'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\DAHS\AppData\Local\Temp\ipykernel_2220\972080183.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hos_data_4['outtime']=hos_data_4.groupby('hadm_id')['outtime'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))


In [210]:
hos_data_4 = hos_data_4[
    (hos_data_4['time'] >= hos_data_4['intime']) &
    (hos_data_4['time'] <= hos_data_4['outtime'])]

In [211]:
hos_data_4

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec,medication
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
1,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-21 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,acetaminophen
2,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sodiumchloride0.9%flush
3,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,insulin
4,20000019.0,2159-03-21 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,docusatesodium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706550,29999723.0,2170-08-09 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2170-08-09 19:50:00,2170-08-10 18:34:46,2170-08-10 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,heparin
706551,29999723.0,2170-08-09 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2170-08-09 19:50:00,2170-08-10 18:34:46,2170-08-10 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,docusatesodium
706552,29999723.0,2170-08-09 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [212]:
hos_data_4=hos_data_4.sort_values(by=['hadm_id','time'])
hos_data_4.reset_index(drop=True,inplace=True)
hos_data_4

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec,medication
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
1,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-21 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,acetaminophen
2,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sodiumchloride0.9%flush
3,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,insulin
4,20000019.0,2159-03-21 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,docusatesodium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661982,29999723.0,2170-08-09 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2170-08-09 19:50:00,2170-08-10 18:34:46,2170-08-10 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,heparin
661983,29999723.0,2170-08-09 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2170-08-09 19:50:00,2170-08-10 18:34:46,2170-08-10 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,docusatesodium
661984,29999723.0,2170-08-09 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [213]:
import pandas as pd

dummies = pd.get_dummies(hos_data_4['medication'], prefix='drug')

dummies = dummies.astype(int)

hos_data_4 = hos_data_4.drop('medication', axis=1)
hos_data_5 = pd.concat([hos_data_4, dummies], axis=1)

In [224]:
hos_data_5

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,drug_warfarin
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-21 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,20000019.0,2159-03-21 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661982,29999723.0,2170-08-09 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2170-08-09 19:50:00,2170-08-1

In [215]:
#hos_data_5.to_csv('pre_hos_v0.3.csv',index= 0)

In [216]:
lab = pd.read_parquet('labevents.parquet')
lab 

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,1,10000032,NaN,45421181,51237,P28Z0X,2180-03-23 11:51:00,2180-03-23 15:15:00,1.4,1.40,None,0.9,1.1,abnormal,ROUTINE,None
1,2,10000032,NaN,45421181,51274,P28Z0X,2180-03-23 11:51:00,2180-03-23 15:15:00,___,15.10,sec,9.4,12.5,abnormal,ROUTINE,VERIFIED.
2,3,10000032,NaN,52958335,50853,P28Z0X,2180-03-23 11:51:00,2180-03-25 11:06:00,___,15.00,ng/mL,30.0,60.0,abnormal,ROUTINE,NEW ASSAY IN USE ___: DETECTS D2 AND D3 25-OH ...
3,4,10000032,NaN,52958335,50861,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,102,102.00,IU/L,0.0,40.0,abnormal,ROUTINE,None
4,5,10000032,NaN,52958335,50862,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,3.3,3.30,g/dL,3.5,5.2,abnormal,ROUTINE,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118171362,118352501,19999987,23865745.0,85842100,51279,None,2145-11-09 05:30:00,2145-11-09 07:06:00,3.52,3.52,m/uL,4.2,5.4,abnormal,ROUTINE,None
118171363,118352502,19999987,23865745.0,85842100,51301,None,2145-11-09 05:30:00,2145-11-09 07:06:00,5.7,5.70,K/uL,4.0,11.0,None,ROUTINE,None
118171364,118352503,19999987,NaN,12592768,50912,P09IS0,2146-02-07 11:13:00,2146-02-07 16:26:00,1.1,1.10,mg/dL,0.4,1.1,None,ROUTINE,None
118171365,118352504,19999987,NaN,12592768,50920,P09IS0,2146-02-07 11:13:00,2146-02-07 16:26:00,None,NaN,None,NaN,NaN,None,ROUTINE,"Using this patient's age, gender, and serum cr..."


In [227]:
lab_1 = lab[lab['hadm_id'].isin(hos_data_5['hadm_id'])][['hadm_id','itemid','charttime','value','valuenum','ref_range_lower','ref_range_upper','flag','priority']]

In [230]:
dic=pd.read_parquet('d_labitems.parquet')
dic

,itemid,label,fluid,category
0,50801,Alveolar-arterial Gradient,Blood,Blood Gas
1,50802,Base Excess,Blood,Blood Gas
2,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas
3,50804,Calculated Total CO2,Blood,Blood Gas
4,50805,Carboxyhemoglobin,Blood,Blood Gas
...,...,...,...,...
1617,53150,Anti Hbs,Blood,Chemistry
1618,53151,Anti-la,Blood,Chemistry
1619,53152,HIV FINAL,Blood,Chemistry
1620,53153,HIV Screen,Blood,Chemistry


In [345]:
lab=pd.read_csv('lab.csv')
lab

,labvalue,abbreviation,itemid_hosp
0,Albumin,Alb,"50862 ,52022, 53085, 53138"
1,Alkaline Phosphate,Alk_Phos,"50863, 53086"
2,Anion Gap,AG,"50868 ,52500"
3,Blood urea nitrogen,BUN,"51006, 52647"
4,Calcium non-ionized,Ca,"50893, 52034, 52035"
5,Creatine Kinase,CK,50910
6,Direct Bilirubin,D_Bil,50883
7,Glucose,Glu,"50809, 50931, 52569,52027"
8,Hematocrit,HCT,"50810, 51221, 51638, 51639, 52028"
9,International Normalized Ratio,INR,"51237, 51675"


In [346]:
itemid_hosp = dict(zip(lab['abbreviation'], lab['itemid_hosp']))

In [366]:
itemid_hosp.keys()

dict_keys(['Alb', 'Alk_Phos', 'AG', 'BUN', 'Ca', 'CK', 'D_Bil', 'Glu', 'HCT', 'INR', 'PH', 'PHOS', 'Platelet', 'Cl', 'SCr', 'Na', 'Potassium', 'T_Bil', 'WBC', 'Gl', 'Mg', 'Ca_ion', 'bicarb', 'AST', 'ALT', 'PTT', 'baseexcess', 'lactate', 'PaO2', 'PaCO2', 'MCHC', 'Red Blood Cells', 'MCV', 'MCH', 'RDW', 'PT', 'Lymphocytes', 'Neutrophils', 'Monocytes', 'Eosinophils', 'Basophils', 'Calcium, Total', 'hemoglobin', 'fio2', 'paco2', 'o2sat', 'crp', 'tropt', 'ck_mb', 'nt_probnp', 'ldh', 'ggt', 'd_dimer', 'fibrinogen', 'thrombin'])

In [347]:
for i in lab['abbreviation'].unique():
    itemid_hosp[i] = [int(item.strip()) for item in itemid_hosp[i].split(',')]

In [353]:
all_values = [item for sublist in itemid_hosp.values() for item in sublist]

In [359]:
lab_1[lab_1['itemid'].isin(all_values)]

,hadm_id,itemid,charttime,value,valuenum,ref_range_lower,ref_range_upper,flag,priority
670,23052089.0,51146,2160-11-20 22:30:00,0.6,0.6,0.0,1.0,None,STAT
671,23052089.0,51200,2160-11-20 22:30:00,2.0,2.0,1.0,7.0,None,STAT
672,23052089.0,51221,2160-11-20 22:30:00,38.2,38.2,40.0,51.0,abnormal,STAT
673,23052089.0,51222,2160-11-20 22:30:00,12.8,12.8,13.7,17.5,abnormal,STAT
674,23052089.0,51244,2160-11-20 22:30:00,33.3,33.3,19.0,53.0,None,STAT
...,...,...,...,...,...,...,...,...,...
118162780,27638769.0,50960,2138-10-09 05:15:00,1.9,1.9,1.6,2.6,None,ROUTINE
118162781,27638769.0,50970,2138-10-09 05:15:00,2.7,2.7,2.7,4.5,None,ROUTINE
118162782,27638769.0,50971,2138-10-09 05:15:00,3.9,3.9,3.3,5.1,None,ROUTINE
118162783,27638769.0,50983,2138-10-09 05:15:00,140,140.0,133.0,145.0,None,ROUTINE


In [360]:
lab_2 = lab_1[lab_1['itemid'].isin(all_values)]

In [370]:
itemid_to_abbr = {value: key for key, value_list in itemid_hosp.items() for value in value_list}
lab_2['lab_name'] = lab_2['itemid'].apply(lambda x: itemid_to_abbr.get(x, 'Unknown'))


C:\Users\DAHS\AppData\Local\Temp\ipykernel_2220\3514776827.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lab_2['lab_name'] = lab_2['itemid'].apply(lambda x: itemid_to_abbr.get(x, 'Unknown'))


In [379]:
lab_2=lab_2[~lab_2['valuenum'].isnull()]

In [383]:
lab_2[~lab_2['ref_range_lower'].isnull()]

,hadm_id,itemid,charttime,value,valuenum,ref_range_lower,ref_range_upper,flag,priority,lab_name
670,23052089.0,51146,2160-11-20 22:30:00,0.6,0.6,0.0,1.0,None,STAT,Basophils
671,23052089.0,51200,2160-11-20 22:30:00,2.0,2.0,1.0,7.0,None,STAT,Eosinophils
672,23052089.0,51221,2160-11-20 22:30:00,38.2,38.2,40.0,51.0,abnormal,STAT,HCT
673,23052089.0,51222,2160-11-20 22:30:00,12.8,12.8,13.7,17.5,abnormal,STAT,hemoglobin
674,23052089.0,51244,2160-11-20 22:30:00,33.3,33.3,19.0,53.0,None,STAT,Lymphocytes
...,...,...,...,...,...,...,...,...,...,...
118162780,27638769.0,50960,2138-10-09 05:15:00,1.9,1.9,1.6,2.6,None,ROUTINE,Mg
118162781,27638769.0,50970,2138-10-09 05:15:00,2.7,2.7,2.7,4.5,None,ROUTINE,PHOS
118162782,27638769.0,50971,2138-10-09 05:15:00,3.9,3.9,3.3,5.1,None,ROUTINE,Potassium
118162783,27638769.0,50983,2138-10-09 05:15:00,140,140.0,133.0,145.0,None,ROUTINE,Na


In [391]:
try:
    pd.to_numeric(lab_2['valuenum'], errors='raise')
    print("All values in 'valuenum' are numeric.")
except ValueError:
    print("There are non-numeric values in 'valuenum'.")

All values in 'valuenum' are numeric.


In [479]:
lab_3 = lab_2[['hadm_id','charttime','valuenum','flag','lab_name']].reset_index(drop=True)

In [480]:
lab_3.isnull().sum()

hadm_id            0
charttime          0
valuenum           0
flag         3235334
lab_name           0
dtype: int64

In [482]:
for lab in lab_3['lab_name'].unique():
    lab_3[lab] = np.nan

In [484]:
for index, row in tqdm(lab_3.iterrows(), total=lab_3.shape[0]):
    lab_3.at[index, row['lab_name']] = row['valuenum']

100%|██████████| 5109834/5109834 [04:58<00:00, 17111.61it/s]


In [485]:
lab_3

,hadm_id,charttime,valuenum,flag,lab_name,Basophils,Eosinophils,HCT,hemoglobin,Lymphocytes,MCH,MCHC,MCV,Monocytes,Neutrophils,Platelet,RDW,Red Blood Cells,WBC,ALT,Alb,Alk_Phos,AG,AST,bicarb,T_Bil,"Calcium, Total",Cl,SCr,Gl,Mg,PHOS,Potassium,Na,BUN,INR,PT,PTT,fibrinogen,thrombin,ldh,nt_probnp,baseexcess,paco2,PH,PaO2,ck_mb,tropt,CK,lactate,crp,D_Bil,fio2,d_dimer,ggt,o2sat
0,23052089.0,2160-11-20 22:30:00,0.6,None,Basophils,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23052089.0,2160-11-20 22:30:00,2.0,None,Eosinophils,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23052089.0,2160-11-20 22:30:00,38.2,abnormal,HCT,NaN,NaN,38.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23052089.0,2160-11-20 22:30:00,12.8,abnormal,hemoglobin,NaN,NaN,NaN,12.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23052089.0,2160-11-20 22:30:00,33.3,None,Lymphocytes,NaN,NaN,NaN,NaN,33.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5109829,27638769.0,2138-10-09 05:15:00,1.9,None,Mg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5109830,27638769.0,2138-10-09 05:15:00,2.7,None,PHOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5109831,27638769.0,2138-10-09 05:15:00,3.9,None,Potassium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5109832,27638769.0,2138-10-09 05:15:00,140.0,None,Na,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [486]:
lab_4 = lab_3.drop(['lab_name','valuenum'], axis=1)
#lab_4= pd.concat([lab_3, dummies_1], axis=1)

In [487]:
lab_4

,hadm_id,charttime,flag,Basophils,Eosinophils,HCT,hemoglobin,Lymphocytes,MCH,MCHC,MCV,Monocytes,Neutrophils,Platelet,RDW,Red Blood Cells,WBC,ALT,Alb,Alk_Phos,AG,AST,bicarb,T_Bil,"Calcium, Total",Cl,SCr,Gl,Mg,PHOS,Potassium,Na,BUN,INR,PT,PTT,fibrinogen,thrombin,ldh,nt_probnp,baseexcess,paco2,PH,PaO2,ck_mb,tropt,CK,lactate,crp,D_Bil,fio2,d_dimer,ggt,o2sat
0,23052089.0,2160-11-20 22:30:00,None,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23052089.0,2160-11-20 22:30:00,None,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23052089.0,2160-11-20 22:30:00,abnormal,NaN,NaN,38.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23052089.0,2160-11-20 22:30:00,abnormal,NaN,NaN,NaN,12.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23052089.0,2160-11-20 22:30:00,None,NaN,NaN,NaN,NaN,33.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5109829,27638769.0,2138-10-09 05:15:00,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5109830,27638769.0,2138-10-09 05:15:00,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5109831,27638769.0,2138-10-09 05:15:00,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5109832,27638769.0,2138-10-09 05:15:00,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [488]:
lab_4.rename(columns={'charttime': 'time'}, inplace=True)

In [489]:
lab_4.rename(columns={'flag': 'abnormal'}, inplace=True)
lab_4['abnormal'] = lab_4['abnormal'].apply(lambda x: 1 if x == 'abnormal' else 0)

In [490]:
lab_4

,hadm_id,time,abnormal,Basophils,Eosinophils,HCT,hemoglobin,Lymphocytes,MCH,MCHC,MCV,Monocytes,Neutrophils,Platelet,RDW,Red Blood Cells,WBC,ALT,Alb,Alk_Phos,AG,AST,bicarb,T_Bil,"Calcium, Total",Cl,SCr,Gl,Mg,PHOS,Potassium,Na,BUN,INR,PT,PTT,fibrinogen,thrombin,ldh,nt_probnp,baseexcess,paco2,PH,PaO2,ck_mb,tropt,CK,lactate,crp,D_Bil,fio2,d_dimer,ggt,o2sat
0,23052089.0,2160-11-20 22:30:00,0,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23052089.0,2160-11-20 22:30:00,0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23052089.0,2160-11-20 22:30:00,1,NaN,NaN,38.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23052089.0,2160-11-20 22:30:00,1,NaN,NaN,NaN,12.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23052089.0,2160-11-20 22:30:00,0,NaN,NaN,NaN,NaN,33.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5109829,27638769.0,2138-10-09 05:15:00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5109830,27638769.0,2138-10-09 05:15:00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5109831,27638769.0,2138-10-09 05:15:00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5109832,27638769.0,2138-10-09 05:15:00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [491]:

#hos_data_5 = hos_data_5.drop(['lab_name','valuenum'], axis=1)
hos_data_6 = pd.concat([hos_data_5, lab_4])

In [494]:
hos_data_6


,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,drug_warfarin,abnormal,Basophils,Eosinophils,HCT,hemoglobin,Lymphocytes,MCH,MCHC,MCV,Monocytes,Neutrophils,Platelet,RDW,Red Blood Cells,WBC,ALT,Alb,Alk_Phos,AG,AST,bicarb,T_Bil,"Calcium, Total",Cl,SCr,Gl,Mg,PHOS,Potassium,Na,BUN,INR,PT,PTT,fibrinogen,thrombin,ldh,nt_probnp,baseexcess,paco2,PH,PaO2,ck_mb,tropt,CK,lactate,crp,D_Bil,fio2,d_dimer,ggt,o2sat
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-21 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [495]:
hos_data_6['intime']=hos_data_6.groupby('hadm_id')['intime'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
hos_data_6['outtime']=hos_data_6.groupby('hadm_id')['outtime'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

C:\Users\DAHS\AppData\Local\Temp\ipykernel_2220\2058053670.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hos_data_6['intime']=hos_data_6.groupby('hadm_id')['intime'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\DAHS\AppData\Local\Temp\ipykernel_2220\2058053670.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hos_data_6['outtime']=hos_data_6.groupby('hadm_id')['outtime'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))


In [496]:
hos_data_6['time']=pd.to_datetime(hos_data_6['time'])

In [497]:
hos_data_6 =hos_data_6[
    (hos_data_6['time'] >= hos_data_6['intime']) &
    (hos_data_6['time'] <= hos_data_6['outtime'])]

In [498]:
hos_data_6=hos_data_6.sort_values(by=['hadm_id','time'])
hos_data_6.reset_index(drop=True,inplace=True)
hos_data_6

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,drug_warfarin,abnormal,Basophils,Eosinophils,HCT,hemoglobin,Lymphocytes,MCH,MCHC,MCV,Monocytes,Neutrophils,Platelet,RDW,Red Blood Cells,WBC,ALT,Alb,Alk_Phos,AG,AST,bicarb,T_Bil,"Calcium, Total",Cl,SCr,Gl,Mg,PHOS,Potassium,Na,BUN,INR,PT,PTT,fibrinogen,thrombin,ldh,nt_probnp,baseexcess,paco2,PH,PaO2,ck_mb,tropt,CK,lactate,crp,D_Bil,fio2,d_dimer,ggt,o2sat
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-21 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [499]:
hos_data_6.to_csv('pre_hos_v0.4.csv',index=0)

In [500]:
hos_data_7 = hos_data_6[hos_data_6['outtime']-hos_data_6['intime']>pd.Timedelta('0 days 09:52:00')]

In [501]:
hos_data_7 = hos_data_7[hos_data_7['outtime']-hos_data_7['intime']<pd.Timedelta('20 days 19:34:07')]

In [502]:
hos_data_7=hos_data_7.sort_values(by=['hadm_id','time'])
hos_data_7.reset_index(drop=True,inplace=True)
hos_data_7

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,drug_warfarin,abnormal,Basophils,Eosinophils,HCT,hemoglobin,Lymphocytes,MCH,MCHC,MCV,Monocytes,Neutrophils,Platelet,RDW,Red Blood Cells,WBC,ALT,Alb,Alk_Phos,AG,AST,bicarb,T_Bil,"Calcium, Total",Cl,SCr,Gl,Mg,PHOS,Potassium,Na,BUN,INR,PT,PTT,fibrinogen,thrombin,ldh,nt_probnp,baseexcess,paco2,PH,PaO2,ck_mb,tropt,CK,lactate,crp,D_Bil,fio2,d_dimer,ggt,o2sat
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-21 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20000019.0,2159-03-21 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [503]:
static_var = hos_data_7.columns[2:37].tolist()+hos_data_7.columns[59:60].tolist()#+hos_data_7.columns[50:-1].tolist()

In [504]:
hos_data_7[static_var] = hos_data_7.groupby('hadm_id')[static_var].ffill().bfill()

In [506]:
hos_data_7.to_csv('pre_hos_v0.5.csv',index=0)

In [512]:
impute = pd.DataFrame(hos_data_7.isnull().sum()).reset_index()
impute

,index,0
0,hadm_id,0
1,time,0
2,race,0
3,gender,0
4,age,0
...,...,...
127,D_Bil,4940035
128,fio2,4941817
129,d_dimer,4941523
130,ggt,4941383


In [515]:
hos_data_7.iloc[:,40:81]=hos_data_7.iloc[:,40:81].fillna(0)

In [517]:
hos_data_7.to_parquet('pre_hos.parquet')